## Overview
- Don't return a value from an expression until it's needed
- Python allocates memory for objects, and it won't be freed as long as the object exists in the program
    - This isn't a problem for small objects, but larger objects require more memory, which can cause performance issues

- Lazy evaluation delays when a program evaluates expressions, which can improve the performance of a program by spreading the time-consuming process across a longer time period
    - It also prevents values that won't be used from being generated

- Errors raised by a lazily-evaluated expression are deferred to later in the program, which can make debugging more difficult

## Examples
`enumerate()` creates an iterator behind the scenes

In [5]:
import random

names = ['Sarah', 'Matt', 'Jim', 'Denise', 'Kate']
random.shuffle(names)

numbered_names = enumerate(names, start=1)
numbered_names

In [6]:
print(next(numbered_names))
print(next(numbered_names))

(1, 'Jim')
(2, 'Kate')


And so does the `zip()` function

In [10]:
names = ['Sarah', 'Matt', 'Jim', 'Denise', 'Kate']
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
random.shuffle(names)

day_name_pairs = zip(weekdays, names)
print(next(day_name_pairs))
print(next(day_name_pairs))

# We can modify the third element in "names" and the third element in "day_name_pairs" will reflect that change since it's evaluated lazily!
names[2] = 'The Coffee Robot'
print(next(day_name_pairs))

# This can be nice, but we need to be cautious as the data is not fixed at the time of creation

('Monday', 'Matt')
('Tuesday', 'Sarah')
('Wednesday', 'The Coffee Robot')


## Itertools

`chain()` - combine 2 iterators into a new iterator, which is then evaluated lazily

In [11]:
import itertools

first_team = ['Sarah', 'Matt', 'Jim']
second_team = ['Denise', 'Kate']

for name in itertools.chain(first_team, second_team):
    print(name)

Sarah
Matt
Jim
Denise
Kate


`islice()`

In [12]:
numbers = [1, 2, 3, 4, 5]
standard_slice = numbers[1:4]
iterator_slice = itertools.islice(numbers, 1, 4)

iterator_slice

In [13]:
numbers[2] = 999

print('standard_slice:', list(standard_slice))
print('iterator_slice:', list(iterator_slice))

standard_slice: [2, 3, 4]
iterator_slice: [2, 999, 4]


## Generator Expressions and Generator Functions
Generator objects are iterators whose values are generated as needed./

In [19]:
coin_toss = (
    'H' if random.random() > 0.5 else 'T'
    for _ in range(10)
)

coin_toss

<generator object <genexpr> at 0x1076e1a40>

In [20]:
print(next(coin_toss))
print(next(coin_toss))

T
T


In [21]:
print(list(coin_toss))  # only 8 tosses left until the generator is exhausted

['T', 'T', 'T', 'H', 'H', 'T', 'T', 'H']


Generator functions can accomplish something similar by using the `yield` keyword

In [18]:
def generate_coin_toss(n):
    for _ in range(n):
        yield 'H' if random.random() > 0.5 else 'T'

coin_toss = generate_coin_toss(10)

print(next(coin_toss))
print(next(coin_toss))
print([el for el in coin_toss])  # again, only 8 tosses left

H
T
['H', 'H', 'T', 'T', 'T', 'T', 'H', 'T']


In [22]:
print(next(coin_toss))  # StopIteration exception when a generator is exhausted

StopIteration: 

## Functional Programming
Functional programming is a paradigm where functions only have access to data input and don't alter the state of objects. Instead, they return new objects. The output of one function is typically passed as the input to the next. Thi smakes it convenient to use lazy evaluation to avoid storing and moving large datasets repeatedly.

`map()`

In [23]:
original_names = ['Sarah', 'Matt', 'Jim', 'Denise', 'Kate']
names = map(str.upper, original_names)
names

In [24]:
print(next(names))
print(list(names))

SARAH
['MATT', 'JIM', 'DENISE', 'KATE']


`filter()`

In [25]:
names = map(str.upper, original_names)  # instantiate a new generator
names = filter(lambda x: 'A' in x, names)
names

In [26]:
print(list(names))

['SARAH', 'MATT', 'KATE']


In [27]:
names = map(str.upper, original_names)  # instantiate a new generator
names = filter(lambda x: 'A' in x, names)
names = filter(lambda x: len(x) == 4, names)  # chain a second filter

print(list(names))

['MATT', 'KATE']


## File Reading Operations
Use the `reader()` function of the `csv` module to evaluate spreadsheet rows lazily by fetching each row only when needed

In [31]:
import csv

file = open('data/superhero_pets.csv', encoding='utf-8', newline='')
data = csv.reader(file)
data

In [32]:
print(next(data))
print(next(data))

['Pet Name', 'Species', 'Superpower', 'Favorite Snack', 'Hero Owner']
['Whiskertron', 'Cat', 'Teleportation', 'Tuna', 'Catwoman']


In [33]:
for row in data:
    print(row)
file.close()

['Flashpaw', 'Dog', 'Super Speed', 'Peanut Butter', 'The Flash']
['Mystique', 'Squirrel', 'Illusion', 'Nuts', 'Doctor Strange']
['Quackstorm', 'Duck', 'Weather Control', 'Bread crumbs', 'Storm']
['Bark Knight', 'Dog', 'Darkness Manipulation', 'Bacon', 'Batman']


## Infinite Data Structures

`itertools.count()`

In [38]:
quarters = itertools.count(start=0, step=0.25)
print(list(itertools.islice(quarters, 0, 5)))
print(list(itertools.islice(quarters, 0, 5)))

[0, 0.25, 0.5, 0.75, 1.0]
[1.25, 1.5, 1.75, 2.0, 2.25]


`itertools.cycle()`

In [39]:
names = ['Sarah', 'Matt', 'Jim', 'Denise', 'Kate']
rota = itertools.cycle(names)
print(list(itertools.islice(rota, 0, 6)))

['Sarah', 'Matt', 'Jim', 'Denise', 'Kate', 'Sarah']


## Advantages
- Reduce memory footprint
- Performance gains in programs with lots of expressions

## Disadvantages
- Debugging may be more difficult